In [ ]:
import cv2

from ultralytics import solutions


def count_objects_in_region(video_path, output_video_path, model_path):
    """Count objects in a specific region within a video."""
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    region_points = [(1726, 1400), (2623, 1408)]
    counter = solutions.ObjectCounter(show=True, region=region_points, model=model_path)

    while cap.isOpened():
        success, im0 = cap.read()
        if not success:
            print("Video frame is empty or processing is complete.")
            break
        results = counter(im0)
        video_writer.write(results.plot_im)

    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()


count_objects_in_region("datasets/video/2252223-uhd_3840_2160_30fps.mp4", "output_video_object_counting.avi", "yolo11n.pt")

In [10]:
import cv2
from ultralytics import YOLO, solutions

def count_objects_in_region(video_path, output_video_path, model_path):
    """Count objects in a specific region within a video and save to file."""
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), "Error reading video file"

    w, h, fps = (int(cap.get(x)) for x in (
        cv2.CAP_PROP_FRAME_WIDTH,
        cv2.CAP_PROP_FRAME_HEIGHT,
        cv2.CAP_PROP_FPS
    ))

    video_writer = cv2.VideoWriter(
        output_video_path,
        cv2.VideoWriter_fourcc(*"mp4v"),
        fps,
        (w, h)
    )

    # Load YOLO OpenVINO model
    ov_model = YOLO(model_path)

    # Object counter dengan OpenVINO
    region_points = [(453, 333), (837, 333)]
    counter = solutions.ObjectCounter(
        show=True,
        region=region_points,
        model=ov_model,
        device="intel:npu"   # bisa "gpu", "GPU.0", atau "AUTO"
    )

    while cap.isOpened():
        success, im0 = cap.read()
        if not success:
            print("Video frame is empty or processing is complete.")
            break

        results = counter(im0)   # Inference OpenVINO
        video_writer.write(results.plot_im)  # Simpan full frame

    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()
    print(f"✅ Output saved to {output_video_path}")

# Jalankan
count_objects_in_region(
    "datasets/video/2165-155327596.mp4",
    "output_video_object_counting_openvino.mp4",
    "yolo11n_openvino_model/"
)

WARNING Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Ultralytics Solutions:  {'source': None, 'model': YOLO(), 'classes': None, 'show_conf': True, 'show_labels': True, 'region': [(453, 333), (837, 333)], 'colormap': 21, 'show_in': True, 'show_out': True, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'figsize': (12.8, 7.2), 'blur_ratio': 0.5, 'vision_point': (20, 20), 'crop_dir': 'cropped-detections', 'json_file': None, 'line_width': 2, 'records': 5, 'fps': 30.0, 'max_hist': 5, 'meter_per_pixel': 0.05, 'max_speed': 120, 'show': True, 'iou': 0.7, 'conf': 0.25, 'device': 'intel:npu', 'max_det': 300, 'half': False, 'tracker': 'botsort.yaml', 'verbose': True, 'data': 'images'}
Loading yolo11n_openvino_model for OpenVINO inference...
Using OpenVINO LATENCY mode for batch=1 inference on (CPU)...
Loading yolo11n_openvino_model for OpenVINO 